Homework 4 \
Center of Mass Position and Velocity \
Reynier Squillace \
Modified from the template by Gurtina Besla 

In [98]:
# import modules
import numpy as np
import astropy.units as u
import pandas as pd
from ReadFile import Read

In [166]:
class CenterOfMass:
# Class to define COM position and velocity properties of a given galaxy 
# and simulation snapshot

    def __init__(self, filename, ptype):
        ''' 
        Class to calculate the 6-D phase-space position of a galaxy's center of mass using
        a specified particle type. 
            
        Inputs:
        -------
                filename (str): snapshot file
                ptype    (int): particle type to use for COM calculations, either 1, 2, or 3
        '''
     
        # read data in the given file using Read
        self.time, self.total, self.data = Read(filename)                                                                                             

        #create an array to store indexes of particles of desired Ptype                                
        self.index = np.where(self.data['type'] == ptype)

        # store the mass, positions, velocities of only the particles of the given type
        # the following only gives the example of storing the mass
        self.m = self.data['m'][self.index]
        # write your own code to complete this for positions and velocities
        self.x = self.data['x'][self.index]
        self.y = self.data['y'][self.index]
        self.z = self.data['z'][self.index]
        self.vx = self.data['vx'][self.index]
        self.vy = self.data['vy'][self.index]
        self.vz = self.data['vz'][self.index]


    def COMdefine(self,a,b,c,m):
        ''' 
        Function to compute the COM of a generic vector quantity by direct weighted averaging.
        
        Inputs:
        -------
            a (float or np.ndarray of floats): first vector component
            b (float or np.ndarray of floats): second vector component
            c (float or np.ndarray of floats): third vector component
            m (float or np.ndarray of floats): particle masses
        
        Returns:
        --------
            a_com (float): first component on the COM vector
            b_com (float): second component on the COM vector
            c_com (float): third component on the COM vector
        '''
        total_mass = np.sum(m)
        # write your own code to compute the generic COM 
        #using Eq. 1 in the homework instructions
        # xcomponent Center of mass
        a_com = np.dot(a, m)/total_mass
        # ycomponent Center of mass
        b_com = np.dot(b, m)/total_mass
        # zcomponent Center of mass
        c_com = np.dot(c, m)/total_mass
        
        # return the 3 components separately
        return a_com, b_com, c_com
       
    
    
    def COM_P(self, delta = 0.1):
        '''
        Method to compute the position of the center of mass of the galaxy 
        using the shrinking-sphere method.

        Inputs:
        -------
                delta (float): error tolerance in kpc. Default is 0.1 kpc
        
        Returns:
        --------
                p_COM (array): 3-D position of the center of mass in kpc
        '''                                                                     

        # Center of Mass Position                                                                                      
        ###########################                                                                                    

        # Try a first guess at the COM position by calling COMdefine                                                   
        x_COM, y_COM, z_COM = self.COMdefine(self.x, self.y, self.z, self.m)
        # compute the magnitude of the COM position vector.
        # write your own code below
        r_COM = np.sqrt(x_COM**2 + y_COM**2 + z_COM**2)


        # iterative process to determine the center of mass                                                            

        # change reference frame to COM frame                                                                          
        # compute the difference between particle coordinates                                                          
        # and the first guess at COM position
        # write your own code below
        x_new = self.x - x_COM
        y_new = self.y - y_COM
        z_new = self.z - z_COM
        r_new = np.sqrt(x_new**2 + y_new**2 + z_new**2)

        # find the max 3D distance of all particles from the guessed COM                                               
        # will re-start at half that radius (reduced radius)                                                           
        r_max = max(r_new)/2.0
        
        # pick an initial value for the change in COM position                                                      
        # between the first guess above and the new one computed from half that volume
        # it should be larger than the input tolerance (delta) initially
        change = 1000.0

        # start iterative process to determine center of mass position                                                 
        # delta is the tolerance for the difference in the old COM and the new one.    
        
        while change > delta:
            # select all particles within the reduced radius (starting from original x,y,z, m)
            # write your own code below (hints, use np.where)
            index2 = np.where(r_new < r_max)
            x2 = x_new[index2]
            y2 = y_new[index2]
            z2 = z_new[index2]
            m2 = self.data['m'][index2]

            # Refined COM position:                                                                                    
            # compute the center of mass position using                                                                
            # the particles in the reduced radius
            # write your own code below
            x_COM2, y_COM2, z_COM2 = self.COMdefine(x2, y2, z2, m2)
            # compute the new 3D COM position
            # write your own code below
            r_COM2 = np.sqrt(x_COM2**2 + y_COM**2 + z_COM**2)

            # determine the difference between the previous center of mass position                                    
            # and the new one.                                                                                         
            change = np.abs(r_COM - r_COM2)
            # uncomment the following line if you want to check this                                                                                               
            # print ("CHANGE = ", change)                                                                                     

            # Before loop continues, reset : r_max, particle separations and COM                                        
            # reduce the volume by a factor of 2 again                                                                 
            r_max /= 2.0
            # check this.                                                                                              
            #print ("maxR", r_max)                                                                                      

            # Change the frame of reference to the newly computed COM.                                                 
            # subtract the new COM
            # write your own code below
            x_new = x2 - x_COM2
            y_new = y2 - y_COM2
            z_new = z2 - z_COM2
            r_new = np.sqrt(x_new**2 + y_new**2 + z_new**2)

            # set the center of mass positions to the refined values                                                   
            x_COM = x_COM2
            y_COM = y_COM2
            z_COM = z_COM2
            r_COM = r_COM2

            
            # create an array (np.array) to store the COM position                                                                                                                                                      
            p_COM = np.array([x_COM, y_COM, z_COM])

        # set the correct units using astropy and round all values
        # and then return the COM positon vector
        # write your own code below
        p_COM = np.round(p_COM, 2)*u.kpc
        return p_COM
        
        
    def COM_V(self, x_COM, y_COM, z_COM):
        ''' 
        Method to compute the center of mass velocity based on the center of mass
        position.

        Inputs:
        -------
                x_COM (kpc): the x component of the center of mass in kpc
                y_COM (kpc): the y component of the center of mass in kpc
                z_COM (kpc): the z component of the center of mass in kpc
            
        Returns:
        --------
                v_COM (array in kpc): 3D velocity of the center of mass in km/s
        '''
        
        # the max distance from the center that we will use to determine 
        #the center of mass velocity                   
        #r_COM = np.sqrt(x_COM.to_value()**2 + y_COM.to_value()**2 +z_COM.to_value()**2)
        rv_max = 15.0 

        # determine the position of all particles relative to the center of mass position (x_COM, y_COM, z_COM)
        # write your own code below
        xV = self.x - x_COM.to_value()
        yV = self.y - y_COM.to_value()
        zV = self.z - z_COM.to_value()
        rV = np.sqrt(xV**2 + yV**2 +zV**2)
        
        # determine the index for those particles within the max radius
        # write your own code below
        indexV = np.where(rV < rv_max)
        
        # determine the velocity and mass of those particles within the max radius
        # write your own code below
        vx_new = self.vx[indexV]
        vy_new = self.vy[indexV]
        vz_new = self.vz[indexV]
        m_new =  self.m[indexV]
        # compute the center of mass velocity using those particles
        # write your own code below
        vx_COM, vy_COM, vz_COM = self.COMdefine(vx_new, vy_new, vz_new, m_new)
        
        # create an array to store the COM velocity
        # write your own code below
        v_COM = np.array([vx_COM, vy_COM, vz_COM])

        # return the COM vector
        # set the correct units using astropy
        # round all values      
        v_COM = np.round(v_COM, 2)*u.km/u.s
        return v_COM                                                                              
     

In [167]:
# Create a Center of mass object for the MW, M31 and M33
MW_COM = CenterOfMass("MW_000.txt", 2)
M31_COM = CenterOfMass('M31_000.txt', 2)
M33_COM = CenterOfMass('M33_000.txt', 2)

In [168]:
MW_COM_p = MW_COM.COM_P()
M31_COM_p = M31_COM.COM_P()
M33_COM_p = M33_COM.COM_P()
MW_COM_v = MW_COM.COM_V(MW_COM_p[0], MW_COM_p[1], MW_COM_p[2])
M31_COM_v = M31_COM.COM_V(M31_COM_p[0], M31_COM_p[1], M31_COM_p[2])
M33_COM_v = M33_COM.COM_V(M33_COM_p[0], M33_COM_p[1], M33_COM_p[2])

/tmp/ipykernel_364/3282904689.py:55: RuntimeWarning: invalid value encountered in double_scalars
  a_com = np.dot(a, m)/total_mass
/tmp/ipykernel_364/3282904689.py:57: RuntimeWarning: invalid value encountered in double_scalars
  b_com = np.dot(b, m)/total_mass
/tmp/ipykernel_364/3282904689.py:59: RuntimeWarning: invalid value encountered in double_scalars
  c_com = np.dot(c, m)/total_mass


In [169]:
df_p = pd.DataFrame([MW_COM_p, M31_COM_p, M33_COM_p], columns = ['x', 'y', 'z'], 
                    index = ['MW', 'M31', 'M33'])
df_v = pd.DataFrame([MW_COM_v, M31_COM_v, M33_COM_v], columns = ['vx', 'vy', 'vz'],
                    index = ['MW', 'M31', 'M33'])
df = pd.concat([df_p, df_v], axis = 1)
df

,x,y,z,vx,vy,vz
MW,-0.0 kpc,0.01 kpc,0.0 kpc,-6.87 km / s,1.24 km / s,-1.25 km / s
M31,-0.01 kpc,-0.01 kpc,0.03 kpc,nan km / s,nan km / s,nan km / s
M33,-0.0 kpc,-0.01 kpc,-0.06 kpc,nan km / s,nan km / s,nan km / s


Unfortunately, my code is not working, and I do not have time to figure out why. I think I understand the shape of what is wrong: somewhere, I have neglected to adjust the disk positions for M31 and M33 with relation to their centers of mass. This is resulting in incorrect center of mass positions *and* preventing the np.where() call from finding any indeces within the specified range. Despite trying to debug it, I haven't been able to, and I know I won't finish it in time.

1. See the DataFrame above, although the values are incorrect for the reasons listed.

2. If I had the right values, I would simply subtract off each coordinate and then find the magnitude of separation. Sadly, there is no real point with the values I have.

3. Same as (2).

4. The iterative process is very important because M31 and M33 may be experiencing significant gravitational disruption due to their proximity. Iterating helps narrow down the particles that actually help describe the center of mass of that particular galaxy, as opposed to particles whose position and velocity have been disrupted by the other galaxy.